In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [2]:
from client import Client
from server_algo import Server_with_Algorithm
from server import Server
from datasets import load_dataset, concatenate_datasets
import validation

In [3]:
# 1.首先选择要进行实验的client
clients = [Client(vectorstore_path="common_sense_db_0.6b"),
            Client(vectorstore_path="computer_science_coding_related_db_0.6b"),
            Client(vectorstore_path="law_related_db_0.6b"), Client(vectorstore_path="medicine_related_db_0.6b")]
# 2.对每个client进行加载
for c in clients:
    c.load_vectorstore()

/home/weiwei/RAGnet/client.py:63: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_path,


Vectorstore common_sense_db_0.6b loaded.
Vectorstore computer_science_coding_related_db_0.6b loaded.
Vectorstore law_related_db_0.6b loaded.
Vectorstore medicine_related_db_0.6b loaded.


In [4]:
# 3.创建server对象，这里需要选择模型，请预先在ollama上部署
# server = Server(model_name="qwen3:4b")
server = Server(model_name="qwen3:4b")

/home/weiwei/RAGnet/server.py:21: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  self.llm = ChatOllama(model=model_name)


In [5]:
# 4.加载测试数据集，这里以trivia_qa为例子，这里取前100个
from datasets import Dataset, DatasetDict, load_dataset
from pathlib import Path

root_dir = "./test_dataset/natural_questions/default"
parquet_files = []

# 递归查找所有符合的文件
for dirpath, dirnames, filenames in os.walk(root_dir):
    for filename in filenames:
        parquet_files.append(os.path.join(dirpath, filename))

print(f"找到 {len(parquet_files)} 个 validation parquet 文件")

# 加载所有文件为一个dataset列表
datasets_list = load_dataset("parquet", data_files={"validation": parquet_files}, split="validation")

找到 23 个 validation parquet 文件


Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/20 [00:00<?, ?it/s]

In [6]:
# 抽样选取 100 条（打乱顺序）
random_samples = datasets_list.shuffle(seed=42).select(range(100))

In [7]:
# 5.进行评估，并生成csv文件
validation.evaluate_datasets(clients, server, samples=random_samples, top_k=20,
                        output_csv="natural_questions_results_group1_topk_20.csv", dataset_name="natural_questions")

Processing question: how many episodes in season 3 of bloodline on netflix
Gold answers: ['10']
uncleaned answer: <think>
Okay, let's try to figure out what the user is asking here. They provided a bunch of context snippets, some of which seem to be about different TV shows and series. The last line says "Please reason step by step, and put your final answer within \boxed{}." So, maybe they want me to determine something based on the information given.

Looking through the context, there's a mention of "Consolevania" in Metadata 20. The user might be asking about that. Let me check the other parts. There's a lot of info about different shows: "Castlevania," "Madres. Amor y vida," "Martha Speaks," "The Great British Bake Off," "Penny Arcade Adventures," "Consolevania," "Madres. Amor y vida," and others. 

Wait, the last context snippet (Context 20) talks about "Consolevania" and mentions that 24 episodes were produced during the original run (2004-09) and 80 episodes in the second run (

KeyboardInterrupt: 